In [ ]:
#! pip install pyaudio

In [1]:
import pyaudio
import wave
import audioop
import whisper

In [2]:
# Recording parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # For speech, 16kHz is usually sufficient
CHUNK = 1024
SILENCE_THRESHOLD = 1000  # Adjust based on your mic sensitivity (lower = more sensitive)
SILENCE_DURATION = 3      # Seconds of silence required to stop recording

In [3]:
def record_until_silence(filename):
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []
    silent_chunks = 0
    required_silent_chunks = int(RATE / CHUNK * SILENCE_DURATION)

    while True:
        data = stream.read(CHUNK)
        frames.append(data)
        # Compute RMS (root mean square) to estimate volume
        rms = audioop.rms(data, 2)
        if rms < SILENCE_THRESHOLD:
            silent_chunks += 1
        else:
            silent_chunks = 0

        if silent_chunks > required_silent_chunks:
            break

    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [4]:
# Record audio from the mic and save it to a file
record_until_silence("recording.wav")

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No

OSError: [Errno -9996] Invalid input device (no default output device)

In [ ]:
# Load and transcribe with Whisper
model = whisper.load_model("turbo")

In [ ]:
# Load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("recording.wav")
audio = whisper.pad_or_trim(audio)

In [ ]:
# Create a log-Mel spectrogram and move it to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

In [ ]:

# Detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# Decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# Print the recognized text
print(result.text)
